In [1]:
import React from 'react';
import Highcharts from 'highcharts/highstock';
import HighchartsReact from 'highcharts-react-official';

const StockChart = ({ data }) => {
  const options = {
    title: {
      text: 'Stock Price'
    },
    xAxis: {
      type: 'datetime'
    },
    yAxis: {
      title: {
        text: 'Price'
      }
    },
    series: [{
      name: 'Stock Price',
      data: data,
      tooltip: {
        valueDecimals: 2
      }
    }]
  };

  return (
    <HighchartsReact
      highcharts={Highcharts}
      constructorType={'stockChart'}
      options={options}
    />
  );
};

export default StockChart;

SyntaxError: invalid syntax (2490503976.py, line 1)

In [ ]:
if (isChatActive) {
return (
<div className={styles.chatPage}>
‹div className=<styles.chatHeader}>
<div className={styles.chatHeaderTitle}> ‹div className=<styles.iconWrapper}>
<FluentIcons.Sparkle24Filled /> < div>
<Text className={styles.chatHeaderText}>{userConfig.buttons[0].label}</Text> < div>
<Text className={styles.chatHeaderDescription}>{userConfig.buttons[0].description}</Text> < div> ‹div className=styles.chatMessages≥>
{chatMessages.map ((message, index : number ) => (
<React.Fragment key={index}>
‹div
className={'${styles.chatMessage} ${
message.role === "user' ? styles.userMessage : styles. assistantMessage
>
{message.role === "user" ? ( message. content
) : (
<div dangerouslySetInnerTML={{ __html: message.content }} />
< div> {subgraphResults && (
‹div className=<styles.subgraphMessage}>
<SubgraphResults results={subgraphResults} /> < div>
{showChart && (
<div className={styles.chartContainer}>
<StockChart data=<stockData? /> < div>
</React .Fragment>
))}
< div>
{isProcessing && <DynamicProgressBar currentStep={currentStep} />} <form onSubmit={handleChatSubmit} className={styles.chatInput}>